# Data Preprocessing for ICENTIA11K Dataset

After exploring the dataset using the notebook [Explore ICENTIA11K Dataset Notebook](./explore_ICENTIA11K_dataset.ipynb), the next step is to preprocess the data. As a reminder, the main sample/label in the dataset is represented by the `p_signal`, which is in the format of `np.ndarray`.

## Preprocessing Steps:

1. **Transform to Tensor:**
   - Convert the `np.ndarray` samples to tensors.

2. **Divide into Features and Labels:**
   - Split the tensor into features (X) and labels (y). The initial split ratio will be 9:1.

3. **Adjusting Sample Length:**
   - To maintain consistency, fix the length of each sample. Split each file into as many examples as possible, ignoring any remainder. It's important to note that this length can be adjusted as a parameter in the script.

4. **Saving Data as Tensors:**
   - For time efficiency, save the preprocessed data as tensors. This helps in quick loading and further analysis without the need for repetitive preprocessing.

### Implementation Details:

To implement these steps, refer to the code in this notebook. Additionally, remember that the sample length is a parameter that can be adjusted based on experimentation. It's worth noting that the script will handle the splitting of files into fixed-length examples during runtime, without creating new files.

This preprocessing stage ensures that the data is in a suitable format for training machine learning models. Adjusting the sample length allows for flexibility in model training and experimentation.



Use [SSSD-main/.../timeseries_utils.py](SSSD-main/docs/instructions/PTB-XL/clinical_ts/timeseries_utils.py) functions to preprocess:

1. ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
2. class TimeseriesDatasetCrops(torch.utils.data.Dataset)

In [2]:
import itertools
import os
import wfdb
from tqdm import tqdm
import pandas as pd
import json
import os
import collections
import numpy as np
import glob
import h5py
import sys
from multiprocessing import Pool

sys.path.append('..')  # Add the parent directory to the sys.path

import utils.data_preparation as data_preparation

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = '6'

# ! gpustat

# data_path = '/mnt/qnap/liranc6/data/'
# subset_data_dir = "/mnt/qnap/liranc6/data/icentia11k-continuous-ecg_normal_sinus_subset/" #patients 0-8

/home/liranc6/miniconda3/envs/ecg/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
def dir_tree(path, level=0):
    for root, _, files in os.walk(path):
        indent = ' ' * level
        print(f'{indent}{root}')
        for file in files:
            print(f'{indent}└─{file}')
        level += 1
        

# dir_tree(subset_data_dir)

In [3]:
import h5py
import numpy as np
import wfdb
from tqdm import tqdm
from utils.data_preparation import find_beat_indices

def extract_and_save_p_signal_to_HDF5(input_dir, output_file, with_R_beats=False):
    """
    Read ECG signals and save the p_signal data as NumPy arrays in an HDF5 file
    while preserving the directory hierarchy.

    Parameters:
    - input_dir: Directory containing ECG data.
    - output_file: The HDF5 file to save the p_signal data.

    Returns:
    - None
    """
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    num_files = sum([1 for _ in os.walk(input_dir)])
    # Open the HDF5 file in write mode
    with h5py.File(output_file, 'w') as h5_file:
        # Traverse the input directory to find data files
        for root, _, files in tqdm(os.walk(input_dir), desc="Processing files", total=num_files, unit="file"):
            for file in files:
                # Check if the file is a data file (ends with .dat)
                if file.endswith('.dat'):
                    record_name = os.path.splitext(file)[0]

                    # Get the relative path from the input directory
                    relative_path = os.path.relpath(root, input_dir)
                    dataset_name = os.path.join(relative_path, f"{record_name}_p_signal")

                    # Read the signal using wfdb
                    filename = os.path.join(root, record_name)
                    
                    signals, fields = wfdb.rdsamp(filename)

                    # Define beat types for annotation plotting
                    beat_types = ['N', 'Q', '+', 'V', 'S']

                    if with_R_beats:
                        # Read the annotations
                        ann = wfdb.rdann(filename, 'atr')
                        indices = [item for sublist in find_beat_indices(ann, beat_types).values() for item in sublist]
                        indices = np.array(indices) - 1
                        # create np array of the same size as the signal and fill it with zeros (default value), put 1 in the indices
                        # of the beats
                        beats = np.zeros(signals.shape[0])
                        beats[indices] = 1

                        # create np array with dims [2, len(signals)] to store the signal and the beats
                        data = np.vstack((signals[:, 0], beats))
                    else:
                        data = signals[:, 0]



                    # Save the p_signal data in the HDF5 file with the dataset name
                    h5_file.create_dataset(dataset_name, data=data)

        

In [ ]:
subset_data_dir = os.path.join(data_path, 'icentia11k-continuous-ecg_normal_sinus_subset')
pSignal_npArray_data_dir_h5 = os.path.join(data_path,
                                           "with_R_beats",
                                           'icentia11k-continuous-ecg_normal_sinus_subset_npArrays.h5'
                                           )
# data_preparation.extract_and_save_p_signal_to_HDF5(subset_data_dir, pSignal_npArray_data_dir_h5, with_R_beats=True)
extract_and_save_p_signal_to_HDF5(subset_data_dir, pSignal_npArray_data_dir_h5, with_R_beats=True)

In [6]:
def print_first_n_datasets_in_HDF5(hdf5_file, n=10):
    """
    Print the first n datasets in an HDF5 file.

    Parameters:
    - hdf5_file: The HDF5 file to read the data from.
    - n: The number of datasets to print.
    """
    with h5py.File(hdf5_file, 'r') as h5_file:
        datasets = []

        def visitor_func(name, node):
            if isinstance(node, h5py.Dataset):
                datasets.append(name)

        h5_file.visititems(visitor_func)
        for dataset in datasets[:n]:
            print(dataset)

# Print the first 10 datasets in the HDF5 file
# print_first_n_datasets_in_HDF5(pSignal_npArray_data_dir_h5, 10)

In [ ]:
def read_dataset_content(hdf5_file, dataset_name):
    """
    Read the content of a specific dataset from an HDF5 file.

    Parameters:
    - hdf5_file: The HDF5 file to read the data from.
    - dataset_name: The name of the dataset to read.
    """
    with h5py.File(hdf5_file, 'r') as h5_file:
        if dataset_name in h5_file:
            data = h5_file[dataset_name][:]
            return data
        else:
            print(f"Dataset {dataset_name} not found in the file.")
            return None

# Read the content of a specific dataset
data = read_dataset_content(pSignal_npArray_data_dir_h5, "p00/p00000/p00000_s00_128028_to_281500_p_signal")
print(f"{data.shape=}")
print(f"{data=}")

In [ ]:
data_preparation.count_items(pSignal_npArray_data_dir_h5)

In [ ]:
data_preparation.print_h5_hierarchy(pSignal_npArray_data_dir_h5)

In [ ]:
import h5py

def split_and_save_data(input_h5_file, window_size, output_h5_file):
    """
    Split each dataset in the input HDF5 file into windows of the specified size
    and save the resulting windows into an output HDF5 file.

    :param input_h5_file: The input HDF5 file with datasets to split.
    :param window_size: The size of each window.
    :param output_h5_file: The output HDF5 file to save the split data.
    :return: None
    """
    def extract_integers(text):
        """
        Extract integers from the given text.

        :param text: The input text containing characters and integers.
        :return: A string containing only the integers found in the text.
        """
        return ''.join(filter(str.isdigit, str(text)))

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_h5_file), exist_ok=True)

    #check if file is being written by another process
    import fcntl

    def is_file_locked(file_path):
        locked = None
        file_object = open(file_path, 'r')
        try:
            fcntl.flock(file_object, fcntl.LOCK_EX | fcntl.LOCK_NB)
            locked = False
        except IOError:
            locked = True
        finally:
            file_object.close()
        return locked

    # Usage
    if is_file_locked(input_h5_file):
        print(f"The file {input_h5_file} is being written by another process.")
    else:
        print(f"The file {input_h5_file} is not locked.")

    print(f"{os.path.exists(input_h5_file)=}, {os.path.exists(output_h5_file)=}")
    with h5py.File(input_h5_file, 'r') as input_file, h5py.File(output_h5_file, 'w') as output_file:
        total_leaf_iterations = 0
        for group_name, group_item in input_file.items():
            assert not isinstance(group_name, h5py.Group), "create only leaf groups"
            for subgroup_name, subgroup_item in tqdm(group_item.items(), desc="Processing Subgroups", unit="subgroup"):
                # print(f"subgroup_name: {subgroup_name}")
                assert not isinstance(subgroup_name, h5py.Group), "leaf groups"
                # dataset_data = []
                total_leaf_iterations += len(subgroup_item)

        progress_bar = tqdm(total=total_leaf_iterations, position=0, desc='Processing')
        for group_name, group_item in input_file.items():
            assert not isinstance(group_name, h5py.Group), "create only leaf groups"
            for subgroup_name, subgroup_item in group_item.items():
                # print(f"subgroup_name: {subgroup_name}")
                assert not isinstance(subgroup_name, h5py.Group), "leaf groups"
                dataset_data = []
                for dataset_name, dataset_item in subgroup_item.items():
                    # print(f"dataset_name: {dataset_name}")
                    assert not isinstance(dataset_name, h5py.Dataset)
                    # Split the dataset into windows
                    data = dataset_item[:] # data.shape = (2, len(signal))
                    num_windows = len(data[1]) // window_size
                    if num_windows > 1:
                        pass
                    
                    # window_data = np.array([data[:, i:i + window_size] for i in range(0, data.shape[1], window_size)])
                    window_data = np.array([data[:, i:i + window_size] for i in range(0, data.shape[1], window_size) if i + window_size <= data.shape[1]])
                    dataset_data.extend(window_data)
                    # Save each window as numpy array and add it to the output dataset
                    # for i in range(num_windows):
                    #     window_data = data[i * window_size: (i + 1) * window_size]
                        
                    #     dataset_data.append(window_data)

                    dataset_name = extract_integers(subgroup_name)
                    progress_bar.update(1)
                output_file.create_dataset(dataset_name, data=dataset_data)
                
# split the arrays to fixed size windows
fs = 250
context_window_size = 9*60*fs  # minutes * seconds * fs
label_window_size = 1*60*fs  # minutes * seconds * fs
window_size = context_window_size+label_window_size

split_pSignal_file = os.path.join(data_path,
                                    "with_R_beats",
                                    'icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits',
                                    '10minutes_window.h5')
base_name, extension = os.path.splitext(os.path.basename(split_pSignal_file))
new_base_name = f"{base_name}_temp{extension}"
temp_filename = os.path.join(os.path.dirname(split_pSignal_file), new_base_name)
# data_preparation.split_and_save_data(pSignal_npArray_data_dir_h5, window_size, split_pSignal_file)
split_and_save_data(pSignal_npArray_data_dir_h5, window_size, split_pSignal_file)

In [ ]:
print_first_n_datasets_in_HDF5(split_pSignal_file, 10)

In [ ]:
data = read_dataset_content(split_pSignal_file, "00000")
print(f"{data.shape=}")
print(f"{data[0].shape=}")
print(f"{data[0][1]=}")
indices = np.where(data[0][1] == 1)
print(f"{len(indices)=}")
print(f"{indices[0].shape=}")
print(f"Indices of 1: {indices[0][:15]}")

In [ ]:
data_preparation.print_h5_hierarchy(split_pSignal_file)
a

In [ ]:
import h5py

def print_top_items(file_path, num_datasets=3, num_items=3):
    with h5py.File(file_path, 'r') as f:
        dataset_counter = 0
        for name, item in f.items():
            if isinstance(item, h5py.Dataset):
                print(f"Dataset: {name}")
                print(f"top {num_items} items:")
                print(item[:num_items])
                dataset_counter += 1
                if dataset_counter >= num_datasets:
                    break

print_top_items(split_pSignal_file)

In [2]:
fs = 250 #sampling rate
SECONDS_IN_MINUTE = 60
# split the arrays to fixed size windows
context_window_size = 10*SECONDS_IN_MINUTE*fs  # minutes * seconds * fs
label_window_size = 5*SECONDS_IN_MINUTE*fs  # minutes * seconds * fs
window_size = context_window_size+label_window_size

# creating subset of normal sinus rhythms (NSR) from the raw data
min_window_size = window_size  # minutes * seconds * sampling rate
# the reason for min_window_size is that I hope to forecast 1-5 minutes ahead.
# and I dont know if a smaller window will give me enough context data
# on top of that, I think I have enough data so I can fiter out the shorter NSR.

data_dir = '/datasets/Icentia11k/physionet.org/files/icentia11k-continuous-ecg/1.0'
my_data_dir = '/home/liranc6/data'

# after creating the subset, with 10 first patients I have more than 10 hours of NSR data
# divided to 62 files of at least 10 minutes each.
# I know its small but I dont need more for now. when I will, I will add more patients. (I used 10/11000 patients)
# subset_data_dir = os.path.join(my_data_dir, 'icentia11k-continuous-ecg_normal_sinus_subset')

pSignal_npArray_data_dir_h5 = os.path.join(my_data_dir, "with_R_beats", 'icentia11k-continuous-ecg_normal_sinus_subset_npArrays.h5')

In [9]:
pSignal_npArray_data_dir_h5

'/home/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays.h5'

In [21]:
import os
import numpy as np
import wfdb
import h5py
from multiprocessing import Pool
import itertools
from tqdm import tqdm


parallel_extract_sinus_rhythms_to_p_signal_array(data_dir, pSignal_npArray_data_dir_h5, min_window_size, start_patient_id=47, end_patient_id=55)

extract_sinus_rhythms_to_new_subset: 100%|██████████| 400/400 [00:25<00:00, 15.95it/s]


In [4]:
path = '/home/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/15_minutes/train/p0_to_p399.h5'

data_preparation.count_items(path)
# data_preparation.read_dataset_content(path, '00000').shape

Dataset: /00000, Count: 90
Dataset: /00001, Count: 50
Dataset: /00002, Count: 32
Dataset: /00003, Count: 14
Dataset: /00004, Count: 15
Dataset: /00005, Count: 43
Dataset: /00006, Count: 35
Dataset: /00007, Count: 91
Dataset: /00008, Count: 41
Dataset: /00009, Count: 13
Dataset: /00011, Count: 65
Dataset: /00012, Count: 10
Dataset: /00013, Count: 126
Dataset: /00014, Count: 155
Dataset: /00015, Count: 75
Dataset: /00016, Count: 82
Dataset: /00017, Count: 13
Dataset: /00018, Count: 159
Dataset: /00019, Count: 75
Dataset: /00020, Count: 20
Dataset: /00021, Count: 30
Dataset: /00022, Count: 81
Dataset: /00023, Count: 35
Dataset: /00024, Count: 52
Dataset: /00025, Count: 38
Dataset: /00026, Count: 16
Dataset: /00027, Count: 72
Dataset: /00028, Count: 69
Dataset: /00029, Count: 104
Dataset: /00030, Count: 32
Dataset: /00031, Count: 31
Dataset: /00032, Count: 34
Dataset: /00033, Count: 29
Dataset: /00034, Count: 64
Dataset: /00035, Count: 97
Dataset: /00036, Count: 67
Dataset: /00038, Count: 